In [ ]:
import sys
import numpy as np
import pandas as pd
import os.path as osp
import os as os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.externals import joblib
from sklearn.datasets import load_digits#
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.utils import class_weight as clsw
from time import time




sys.path.append("../")
from parse_fasta import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
def handle_data(df):
    print('Shape data{}'.format(df.shape))
    
    # transform labels to numerical
    y = df['label'].values
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)
    
    print('Number of unique classes and their counts {}'.format(np.unique(y, return_counts=True)))
    #add ML ready labels to data frame

    #STEP 1: remove all classes with occurences less then threshold
    thresh = 10
    _, counts = np.unique(y, return_counts=True)
    mask = np.isin(y, np.where(counts >= thresh)[0])
    print(np.sum(counts[counts < 10]) == len(df) - len(df[mask]))
    df = df[mask]
    
    # STEP 2: remove datapoints where kmer frequency sum is less or equal to threshold
    ths_kmer = 0
    mask = np.sum(df.iloc[:, 2:].values, axis=1) > ths_kmer
    df = df[mask]
    print('New data shape: {}'.format(df.shape))
    
        
    # transform labels to numerical
    y = df['label'].values
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)
    
    X = df.iloc[:, 2:].values

    print('New shape after removing classes{}'.format(df.shape))
    
    return X,y
    
    

In [ ]:
def random_benchmarks(y_test):
    unique, counts = np.unique(y_test, return_counts=True)

    #computing uniformly (random guessing) class predictions
    n_samples = len(y_test)
    n_classes = len(unique)
    uniform_preds = np.random.randint(n_classes, size=n_samples)

    #computing class predictions using Naive bayes(i.e. class probs from data)
    class_probs = counts/n_samples
    naive_preds = np.random.choice(unique, size=n_samples, p=class_probs)

    return accuracy_score(y_test, uniform_preds), accuracy_score(y_test, naive_preds)

In [ ]:
def bootstrap(clf2 ,X_test ,y_test, b):
    '''
    :param X: Input test data
    :param y: test labels
    :param b: number of bootstrap samples to draw (100-1000)
    :return: the standard deviation (std) aver all and per class over all bootstrap samples.
    '''

    #initilize lists to store elements
    X_test = pd.DataFrame(X_test)
    y_test = pd.DataFrame(y_test).iloc[:,0]
    
    acc_list = list()
    classes = pd.unique(y_test)

    # compute length of data set
    n = len(X_test)

    for i in range(b):

        # generate random indices and draw sample
        X_sub = X_test.sample(int(n*0.6))
        y_sub = y_test[X_sub.index]

        # compute accurace and append to list
        acc = accuracy_score(y_sub, clf2.predict(X_sub))
        acc_list.append(acc)


    acc_array = np.array(acc_list)
    sd_acc = np.std(acc_array)

    return sd_acc

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def presentation_metric(clf, X_test, y_test):
    
    # Evaluate model pipeline on test data
    y_pred = clf.predict(X_test)
    overall_acc = accuracy_score(y_test, y_pred)
    sd_acc = bootstrap(clf, X_test, y_test, 100)

    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)


    class_names = np.unique(y)
    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title='Confusion matrix, without normalization', cmap=plt.cm.Blues)
    
    random, naive = random_benchmarks(y_test)
    
    return overall_acc, sd_acc, cnf_matrix , random , naive

In [ ]:
def return_opt_model(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                       test_size=0.2,random_state=7, stratify = y)
    
                                                    
    #build a random forest classifier
    clf = RandomForestClassifier(n_estimators=30,  class_weight = "balanced_subsample")

    # specify parameters and distributions to sample from
    param_dist = {"max_depth": [3, None],
                  "max_features": [None, 'sqrt', 'log2'],
                  "min_samples_split": sp_randint(2, 11),
                  "min_samples_leaf": sp_randint(1, 11),
                  "criterion": ["gini", "entropy"]}

    # run randomized search
    n_iter_search = 30
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                       n_iter=n_iter_search,cv=5, verbose=1, n_jobs=64, 
                                       refit='accuracy', scoring='accuracy')

    start = time()
    random_search.fit(X_train, y_train)
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
          " parameter settings." % ((time() - start), n_iter_search))
    
    best_estimator = random_search.best_estimator_
    
    return best_estimator, X_test, y_test

# Get reuslts presentation Eukaryots

In [ ]:
data_dir = '/home/olle/Documents/University Courses/Protein Prediction II/ppcs2-project-master/dataset'
data_file = data_dir + '/subloc_k3_s5_eukaryota.1682.fa_1516057206692442.csv'

In [ ]:
#Load data
df = pd.read_csv(data_file, sep=',')
print(df.shape)

#create ML ready data
X,y = handle_data(df)

In [ ]:
#train and tune optimál model
best_model,X_test, y_test = return_opt_model(X,y)

In [ ]:
# Save model for future use
joblib.dump(best_model, 'rf_model_03_eukaryota')

# Load model
clf2 = joblib.load('rf_model_03_eukaryota')

In [ ]:
#get metrics for presentation
overall_acc, sd_acc, cnf_matrix, random_acc, naive_acc = presentation_metric(clf2, X_test, y_test)

# Get Results Archea

In [ ]:
data_dir = '/home/olle/Documents/University Courses/Protein Prediction II/ppcs2-project-master/dataset'
data_file = data_dir + '/subloc_k3_s5_archaea.59.fa_1516096802270404.csv'

In [ ]:
#Load data
df = pd.read_csv(data_file, sep=',')
print(df.shape)

#create ML ready data
X,y = handle_data(df)

In [ ]:
#train and tune optimál model
best_model,X_test, y_test = return_opt_model(X,y)

In [ ]:
# Save model for future use
joblib.dump(best_model, 'rf_model_03_archea')

# Load model
clf2 = joblib.load('rf_model_03_archea')

In [ ]:
#get metrics for presentation
overall_acc, sd_acc, cnf_matrix, random_acc, naive_acc = presentation_metric(clf2, X_test, y_test)

In [ ]:
overall_acc,sd_acc,cnf_matrix,random_acc,naive_acc

# Get Results Bacteria

In [ ]:
data_dir = '/home/olle/Documents/University Courses/Protein Prediction II/ppcs2-project-master/dataset'
data_file = data_dir + '/subloc_k3_s5_bacteria.479.fa_1516097171505748.csv'

In [ ]:
#Load data
df = pd.read_csv(data_file, sep=',')
print(df.shape)

#create ML ready data
X,y = handle_data(df)

In [ ]:
#train and tune optimál model
best_model,X_test, y_test = return_opt_model(X,y)

In [ ]:
# Save model for future use
joblib.dump(best_model, 'rf_model_03_bacteria')

# Load model
clf2 = joblib.load('rf_model_03_bacteria')

In [ ]:
#get metrics for presentation
overall_acc, sd_acc, cnf_matrix, random_acc, naive_acc = presentation_metric(clf2, X_test, y_test)

In [ ]:
overall_acc,sd_acc,cnf_matrix,random_acc,naive_acc